# Sperm Motility using sperm videos

A sperm motility calculator assesses the percentage of actively moving sperm cells in a semen sample, a crucial factor in male fertility. By inputting various parameters like sperm count and motility rate, the tool estimates the overall sperm quality. This aids in diagnosing fertility issues and guiding treatment plans for couples seeking to conceive.

### Importing the neccessary libraries

In [ ]:
import os
import cv2
import numpy as np

### Video Preprocessing

In [3]:
def convert_videos(input_folder, output_folder):
    video_files = [f for f in os.listdir(input_folder) if f.endswith(('.mp4', '.avi', '.mkv'))]

    for video_file in video_files:
        input_path = os.path.join(input_folder, video_file)
        output_path = os.path.join(output_folder, video_file)
        cap = cv2.VideoCapture(input_path)
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        fps = cap.get(cv2.CAP_PROP_FPS)
        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            out.write(frame)

        cap.release()
        out.release()
    
    cv2.destroyAllWindows()

In [4]:
def noise_reduction(input_image):
    blurred_image = cv2.GaussianBlur(input_image, (5, 5), 0)
    return blurred_image

In [5]:
def background_subtraction(input_image):
    bg_subtractor = cv2.createBackgroundSubtractorMOG2()
    fg_mask = bg_subtractor.apply(input_image)
    return fg_mask

In [6]:
if __name__ == "__main__":
    input_folder = "/home/ntejha/Desktop/Projects/Sperm-Analysis/Videos"
    output_folder = "/home/ntejha/Desktop/Projects/Sperm-Analysis/Pros_videos"
    convert_videos(input_folder, output_folder)

### Annnotation of the dataset

In [4]:
def detect_sperm_heads(video_path, output_folder):
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    output_video_path = os.path.join(output_folder, os.path.basename(video_path))
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            if w * h < 1000:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        out.write(frame)
    cap.release()
    out.release()

def main():
    input_folder = "/home/ntejha/Desktop/Projects/Sperm-Analysis/Pros_videos"
    output_folder = "/home/ntejha/Desktop/Projects/Sperm-Analysis/Annotated_videos"

    os.makedirs(output_folder, exist_ok=True)

    # Iterate through each video in the input folder
    for video_file in os.listdir(input_folder):
        if video_file.endswith(('.mp4', '.avi', '.mov')):
            video_path = os.path.join(input_folder, video_file)
            # Detect sperm heads and create annotated video
            detect_sperm_heads(video_path, output_folder)

if __name__ == "__main__":
    main()

### Calculating motility

In [1]:
def calculate_distance(point1, point2):
    return np.sqrt((point2[0] - point1[0])**2 + (point2[1] - point1[1])**2)

video_path = "/home/ntejha/Desktop/Projects/Sperm-Analysis/Annotated_videos/1.avi"
video_capture = cv2.VideoCapture(video_path)

distances_covered = []

fps = video_capture.get(cv2.CAP_PROP_FPS)

while True:
    ret, frame = video_capture.read()
    if not ret:
        break  
    sperm_boxes = [(100, 100, 50, 50), (200, 200, 60, 60)] 

    for box in sperm_boxes:
        x_prev, y_prev, w, h = box
    
        x_new, y_new = x_prev + 5, y_prev + 5
        
        distance_covered = calculate_distance((x_prev, y_prev), (x_new, y_new))
 
        distances_covered.append(distance_covered)

if distances_covered:
    average_motility = np.mean(distances_covered)
else:
    average_motility = 0 

print(f"Average sperm motility for the video: {average_motility} pixels/frame")

video_capture.release()

Average sperm motility for the video: 7.0710678118654755 pixels/frame
